# Module for sampling datas with 1Hz 

In [2]:
import os
import sys
import time
import serial
import math
import numpy as np
from tqdm import tqdm
from statistics import mean
import pandas as pd
from scipy.interpolate import interp1d
from local_occupancy import sensor_measurements, sensor_distances
from local_occupancy import thymio_coords, sensor_pos_from_center, sensor_angles
%matplotlib inline

sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

test_functions = False

th = Thymio.serial(port="COM8", refreshing_rate=0.1)

time.sleep(3) # To make sure the Thymio has had time to connect

variables = th.variable_description()

for var in variables : 
    print(var)

KeyboardInterrupt: 

# Setting Timer

A basic timer function 

In [ ]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False


# Function to get real time values
- Function is called with f=1Hz
- proximity, acceleraation and motors' speed values are stored in global variables

In [ ]:
from time import sleep
Ts=1
value_proximity=0
value_acceleration=0
value_speed=0

In [ ]:
def print_sensor_values():
    global value_proximity
    global value_acceleration
    global value_speed
    value_proximity=th['prox.horizontal']
    value_acceleration=th['acc']
    value_speed=[th['motor.left.speed'],th['motor.right.speed']]
    for i in range(2):
        if value_speed[i]>600:
            value_speed[i]=value_speed[i]-2**16
    for i in range(3):
        if value_acceleration[i]>600:
            value_acceleration[i]=value_acceleration[i]-2**16
    
    
    
    #print("proximity: ",value_proximity)
    print("accelaration: ",value_acceleration)
    #print("motors' speed: ",value_speed)


#print_sensor_values('prox.horizontal')

# Call it to start the timer 

In [ ]:
rt = RepeatedTimer(Ts, print_sensor_values)

# NOT IMPORTANT PART
- In case you wanna see the measurement datas from Thymio, try it then :) 

In [ ]:
def move(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    # Printing the speeds if requested
    if verbose: print("\t\t Setting speed : ", l_speed, r_speed)
    
    # Changing negative values to the expected ones with the bitwise complement
    l_speed = l_speed if l_speed>=0 else 2**16+l_speed
    r_speed = r_speed if r_speed>=0 else 2**16+r_speed

    # Setting the motor speeds
    th.set_var("motor.left.target", l_speed)
    th.set_var("motor.right.target", r_speed)

In [ ]:
move(-100,100,True)

In [ ]:
move(0,0,True)